# CPU vs. GPU Scikit-Learn Benchmarks

## Instructions
1. Please rename the "YOUR_SYSTEM" folder, that contains this notebook, into a system name of your choice.
2. If you'd like to compare your system to mine, please do not change the parameter names and values when calling the benchmarking functions.
3. If you'd like to compare 2 or 3 of your own systems and disregard the "LEGION" and "ROG" systems, please feel free to customize the parameter names (n_features, max_features, n_estimators).
4. Please run all notebook cells up to: `6. GPU Benchmarking`
5. Once you are done with the CPU benchmarks, and the results CSV file is saved in the current directory, you must restart the kernel, at: `6.1 Restart Kernel`.
6. After you restart the kernel, make sure to load cuML Scikit-learn first `6.2. Load cuML Scikit-Learn GPU Extenssion`.
7. Then, run:
   - section 2. Imports
   - section 3. Load Dataset
   - section 4. Benchmarking Functions
   - section 6.4. GPU Results and onward
8. Once you have both a "cpu_results.csv" and a "gpu_results.csv", you can then navigate back to the "speedtest directory" and run the visualization notebook: speedtest_results_analysis.ipynb

## Imports

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures
import time
import pandas as pd
import subprocess
from IPython import get_ipython
import pynvml

## Load a Dataset of Your Choice

In [ ]:
covertype = datasets.fetch_covtype()
x = covertype.data
y = covertype.target

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=432
)

print("columns:", x.shape[1])
print("rows:", x.shape[0])

## Benchmarking Functions

In [ ]:
def benchmark_one(
    x_train, x_test, y_train, y_test,
    random_state=8, 
    n_jobs=-1, 
    n_estimators=100, 
    max_features="sqrt", 
    poly=False
):
    """
    training benchmark function for CPU and GPU
    """
    if poly == True:
        poly = PolynomialFeatures()
        x_train = poly.fit_transform(x_train)
        x_test = poly.fit_transform(x_test)

    # CPU 1540 features
    model = RandomForestClassifier(
        random_state=random_state,
        n_jobs=n_jobs,
        n_estimators=n_estimators,
        max_features=max_features,     
        )

    start_time = time.time()
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    n_features = x_train.shape[1]
    end_time = time.time() - start_time

    return [
        n_estimators, 
        max_features, 
        random_state, 
        n_features, 
        accuracy, 
        end_time
    ]

def benchmark_many(
    estimators_many, max_features_many, file_name
):
    """
    Hyperparameter tuning function for CPU and GPU
    """
    df = pd.DataFrame(
        columns=[
            "n_estimators", 
            "max_features", 
            "random_state", 
            "n_features", 
            "accuracy", 
            "time"
        ]
    )

    idx = 0
    for e in estimators_many:
        for m in max_features_many:
            for p in [True, False]:
                df.loc[idx] = benchmark_one(
                    x_train, 
                    x_test, 
                    y_train, 
                    y_test, 
                    poly=p, 
                    n_estimators=e, 
                    max_features=m
                )
                print(
                    idx, 
                    "| poly:", p, 
                    "| max_features:", m, 
                    "| trees:",e , 
                    "| time:", df.loc[idx]["time"]
                )
                idx += 1
                
                # cool down for 5 seconds
                time.sleep(5)       
                # save progress
                df.to_csv(file_name)
                
    print("benchmarking complete, saved as", file_name)
    return df

## CPU Benchmarking
The following cell will find the name of the CPU and print its processing time over different parameter combinations.

In [ ]:
device_make = subprocess.check_output("cat /proc/cpuinfo | grep 'model name' | head -n 1", shell=True, text=True).split(":")[1].strip()
print("wait while benchmarking CPU:", device_make, "...")

df = benchmark_many([100, 250, 500], ["sqrt", 1.0], "cpu_results.csv")

### CPU Results

In [ ]:
df

## GPU Benchmarking

To perform GPU benchmarking, you must follow the instructions (detailed information in the instructions section above, in the first markdown cell)

### Restart Kernel

In [ ]:
get_ipython().kernel.do_shutdown(restart=True)

### Load cuML Scikit-Learn GPU Extenssion

In [ ]:
import warnings
warnings.filterwarnings('ignore')

%load_ext cuml.accel

### 4.2 Re-Run Sections 1-3

Please make sure you:
- import required modules
- load dataset
- define nescessary functions

### GPU Results
The following cell will find the name of the GPU and print its processing time over different parameter combinations.

In [ ]:
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_make = pynvml.nvmlDeviceGetName(handle) 
pynvml.nvmlShutdown()

print("wait while benchmarking GPU:", device_make, "...")

df = benchmark_many([100, 250, 500], ["sqrt", 1.0], "gpu_results.csv")
df